# 🏠 부동산 실거래가 예측 대회 - KKH - test 데이터 전처리
> - test 데이터의 결측치를 처리한다.
> - 몇 개 피처를 생성하였다.
> - 서울시 공동주택 아파트 정보를 활용한다.
> - [서울시 공동주택 아파트 정보](https://data.seoul.go.kr/dataList/OA-15818/S/1/datasetView.do)
> - kimkihong / helpotcreator@gmail.com / Upstage AI Lab 3기
> - 2024.07.16.화 ~ 2024.07.19.금 19:00

## 라이브러리 & 폰트 설정

- 폰트는 .otf 파일을 직접 위치시켜서 임포트 시켰다.
- 본인은 우분투와 윈도우 두 환경에서 동시에 작업 중인데, 이와 같이 폰트를 설정하면, 문제 없다.

In [44]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(fname=r'font/NanumGothic.otf', name='NanumBarunGothic')
fm.fontManager.ttflist.insert(0, fe)
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'})
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance


# 모든 열을 표시하도록 설정
pd.set_option('display.max_columns', None)

In [45]:
test = pd.read_csv('data/test.csv', encoding='utf-8')
gps = pd.read_csv('data/jaemyung_newXY_for_test.csv', encoding='utf-8')
add = pd.read_csv('data/yoonjae_add.csv', encoding='EUC-KR') # 서울시 공동주택 아파트 정보
subway = pd.read_csv('data/subway_feature.csv', encoding='utf-8')

In [46]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9272 entries, 0 to 9271
Data columns (total 51 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   시군구                     9272 non-null   object 
 1   번지                      9270 non-null   object 
 2   본번                      9272 non-null   float64
 3   부번                      9272 non-null   float64
 4   아파트명                    9262 non-null   object 
 5   전용면적(㎡)                 9272 non-null   float64
 6   계약년월                    9272 non-null   int64  
 7   계약일                     9272 non-null   int64  
 8   층                       9272 non-null   int64  
 9   건축년도                    9272 non-null   int64  
 10  도로명                     9272 non-null   object 
 11  해제사유발생일                 212 non-null    float64
 12  등기신청일자                  9272 non-null   object 
 13  거래유형                    9272 non-null   object 
 14  중개사소재지                  9272 non-null   

## test 데이터의 결측치를 확인해본다.

In [47]:
test.isna().mean() * 100

시군구                        0.000000
번지                         0.021570
본번                         0.000000
부번                         0.000000
아파트명                       0.107852
전용면적(㎡)                    0.000000
계약년월                       0.000000
계약일                        0.000000
층                          0.000000
건축년도                       0.000000
도로명                        0.000000
해제사유발생일                   97.713546
등기신청일자                     0.000000
거래유형                       0.000000
중개사소재지                     0.000000
k-단지분류(아파트,주상복합등등)        70.987921
k-전화번호                    70.923210
k-팩스번호                    71.246764
단지소개기존clob                94.025022
k-세대타입(분양형태)              70.772217
k-관리방식                    70.772217
k-복도유형                    70.793788
k-난방방식                    70.772217
k-전체동수                    70.933995
k-전체세대수                   70.772217
k-건설사(시공사)                70.955565
k-시행사                     70.966350
k-사용검사일-사용승인일             70

## '아파트명' 피처의 결측치를 확인한다.

In [48]:
test[test['아파트명'].isna()]

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일
2451,서울특별시 구로구 구로동,740-7,740.0,7.0,NaN,35.10,202308,21,4,1996,구로동로12길 49,NaN,,중개거래,서울 구로구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2452,서울특별시 구로구 구로동,743-27,743.0,27.0,NaN,59.56,202309,8,1,1994,구로동로22길 52-2,NaN,20230915,중개거래,서울 구로구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2453,서울특별시 구로구 구로동,747-34,747.0,34.0,NaN,49.11,202308,4,2,2000,도림로3길 35-5,NaN,20230828,직거래,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2454,서울특별시 구로구 구로동,752-17,752.0,17.0,NaN,33.56,202307,3,3,1994,구로동로22길 76-6,NaN,20230727,중개거래,서울 구로구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2455,서울특별시 구로구 구로동,780-86,780.0,86.0,NaN,35.55,202307,5,5,1961,도림로12길 11,NaN,20230706,직거래,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4035,서울특별시 동대문구 장안동,404-13,404.0,13.0,NaN,84.86,202308,18,7,2005,천호대로77길 62,NaN,20230822,직거래,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4449,서울특별시 동작구 상도동,323-4,323.0,4.0,NaN,106.51,202307,5,3,2005,국사봉1길 18,NaN,,중개거래,서울 동작구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8041,서울특별시 영등포구 대림동,1101-1,1101.0,1.0,NaN,14.46,202307,8,6,2012,도림천로19길 12,NaN,20230809,중개거래,서울 영등포구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8042,서울특별시 영등포구 대림동,1101-1,1101.0,1.0,NaN,14.46,202307,21,10,2012,도림천로19길 12,NaN,,중개거래,서울 영등포구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8966,서울특별시 중구 신당동,432-904,432.0,904.0,NaN,59.16,202309,13,4,2001,동호로11마길 20-8,NaN,,중개거래,서울 중구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## '아파트명' 피처의 10개 결측치 처리

In [49]:
# 네이버 지도에도 나오지 않는 빌딩 건물들
test.loc[(test['도로명'] == '구로동로12길 49') & (test['층'] == 4) & (test['아파트명'].isna()), '아파트명'] = '구로동로12길 49'
test.loc[(test['도로명'] == '도림로3길 35-5') & (test['층'] == 2) & (test['아파트명'].isna()), '아파트명'] = '도림로3길 35-5'
test.loc[(test['도로명'] == '구로동로22길 76-6') & (test['층'] == 3) & (test['아파트명'].isna()), '아파트명'] = '구로동로22길 76-6'
test.loc[(test['도로명'] == '도림로12길 11') & (test['층'] == 5) & (test['아파트명'].isna()), '아파트명'] = '도림로12길 11'

# 네이버 지도에서 직접 찾은 아파트
test.loc[(test['도로명'] == '구로동로22길 52-2') & (test['층'] == 1) & (test['아파트명'].isna()), '아파트명'] = '계영주택'
test.loc[(test['도로명'] == '천호대로77길 62') & (test['층'] == 7) & (test['아파트명'].isna()), '아파트명'] = '아이엔에스새터아파트'
test.loc[(test['도로명'] == '국사봉1길 18') & (test['층'] == 3) & (test['아파트명'].isna()), '아파트명'] = '상진아파트'
test.loc[(test['도로명'] == '도림천로19길 12') & (test['층'] == 6) & (test['아파트명'].isna()), '아파트명'] = '유탑유블레스'
test.loc[(test['도로명'] == '도림천로19길 12') & (test['층'] == 10) & (test['아파트명'].isna()), '아파트명'] = '유탑유블레스'
test.loc[(test['도로명'] == '동호로11마길 20-8') & (test['층'] == 4) & (test['아파트명'].isna()), '아파트명'] = '성민아트'

## '아파트명' 길이가 비정상적으로 짧은 데이터

- 없다.
- 참고로, 길이가 짧은 '은마', '한신' 아파트명이 존재한다.

In [50]:
test[test['아파트명'].str.len() < 2]

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일


## '번지' 피처의 결측치 처리

In [51]:
test[test['번지'].isna()]

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일
4963,서울특별시 서초구 내곡동,NaN,0.0,0.0,서초포레스타2단지,84.48,202308,19,12,2015,헌릉로8길 45,NaN,,중개거래,서울 서초구,아파트,0220579560,0220579562,NaN,기타,위탁관리,복도식,개별난방,13.0,1077.0,고려개발,에스에치공사,2015-06-18 00:00:00.0,134431.0,59281.0,134431.0,930.0,147.0,0.0,NaN,NaN,NaN,2023-09-23 16:53:27.0,NaN,위탁,단일계약,위탁,8252.0,1185.0,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0
4964,서울특별시 서초구 내곡동,NaN,0.0,0.0,서초포레스타2단지,84.48,202308,22,11,2015,헌릉로8길 45,NaN,,중개거래,서울 서초구,아파트,0220579560,0220579562,NaN,기타,위탁관리,복도식,개별난방,13.0,1077.0,고려개발,에스에치공사,2015-06-18 00:00:00.0,134431.0,59281.0,134431.0,930.0,147.0,0.0,NaN,NaN,NaN,2023-09-23 16:53:27.0,NaN,위탁,단일계약,위탁,8252.0,1185.0,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0


## '번지' 피처의 2개 결측치 처리

In [52]:
# 네이버 지도에서 도로명으로 검색해보면, 지번: '내곡동 384'로 적혀 있음
test.loc[(test['도로명'] == '헌릉로8길 45') & (test['층'] == 12) & (test['번지'].isna()), '번지'] = '384'
test.loc[(test['도로명'] == '헌릉로8길 45') & (test['층'] == 11) & (test['번지'].isna()), '번지'] = '384'

## '전용면적(㎡)' 피처의 이상치 확인

- 결측치는 이미 없음을 알고 있기에, 추가 확인하지 않았다.
- 전용면적이 엄청 작은 경우를 찾아보았지만, 없다.

In [53]:
test[test['전용면적(㎡)'] < 10.0]

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일


## test 데이터 전체 결측치 확인(중간 확인)

In [54]:
test.isna().mean() * 100

시군구                        0.000000
번지                         0.000000
본번                         0.000000
부번                         0.000000
아파트명                       0.000000
전용면적(㎡)                    0.000000
계약년월                       0.000000
계약일                        0.000000
층                          0.000000
건축년도                       0.000000
도로명                        0.000000
해제사유발생일                   97.713546
등기신청일자                     0.000000
거래유형                       0.000000
중개사소재지                     0.000000
k-단지분류(아파트,주상복합등등)        70.987921
k-전화번호                    70.923210
k-팩스번호                    71.246764
단지소개기존clob                94.025022
k-세대타입(분양형태)              70.772217
k-관리방식                    70.772217
k-복도유형                    70.793788
k-난방방식                    70.772217
k-전체동수                    70.933995
k-전체세대수                   70.772217
k-건설사(시공사)                70.955565
k-시행사                     70.966350
k-사용검사일-사용승인일             70

## GPS 좌표 결측치 해결

- 팀원께서 네이버지도API로 찾아낸 GPS 좌표로 교체함.

In [55]:
test = pd.concat([test, gps], axis=1)
test = test.drop(['좌표X', '좌표Y'], axis=1)
test = test.rename(columns={'좌표X_2': '좌표X'})
test = test.rename(columns={'좌표Y_2': '좌표Y'})

## '시군구' 피처를 이용하여 '구'와 '동' 피처로 나누고, 추가한다.

- '시'는 서울특별시 데이터만 존재하기에 피처 추가하지 않는다.
- 본 과정은 EDA 부분에서 진행할 예정이었으나, 본 과정(외부 데이터 연동)에서 '구' 피처가 필요하여 미리 진행한다.

In [56]:
test['구'] = test['시군구'].apply(lambda x: x.split()[1])
test['동'] = test['시군구'].apply(lambda x: x.split()[2])

## 데이터를 합쳐줄, 함수 작성

- train 데이터와 add 데이터가 비슷하게 보이지만, 피처명이 다른게 많다.
- 하지만 다행히, 결측치가 많아서 채워넣어야할 뒷 부분은 피처명이 서로 같다.
- k-전화번호 피처를 포함해서 그 이후는 순서도 서로 같고, 피처명도 서로 같다.

In [57]:
# 결측치 확인하고, 채워 넣을 피쳐 리스트 작성
missing_check_list = ['k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
              'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
              'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
              'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
              'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
              '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드', '단지신청일']

def missing_update(key_feature: str):
    # 결측치 채우기 전 결측치 수 및 비율 확인
    total_rows = len(test)
    missing_before = test[missing_check_list].isnull().sum().reset_index()
    missing_before.columns = ['Feature', 'Missing_Before']
    missing_before['Missing_Before_%'] = (missing_before['Missing_Before'] / total_rows) * 100

    # 결측치 채우기
    for feature in missing_check_list:
        if feature in add.columns:
            add_feature_dict = add.set_index(key_feature)[feature].to_dict()
            test[feature] = test.apply(lambda row: add_feature_dict.get(row[key_feature], row[feature]) if pd.isnull(row[feature]) else row[feature], axis=1)

    # 결측치 채우기 후 결측치 수 및 비율 확인
    missing_after = test[missing_check_list].isnull().sum().reset_index()
    missing_after.columns = ['Feature', 'Missing_After']
    missing_after['Missing_After_%'] = (missing_after['Missing_After'] / total_rows) * 100

    # 결측치 비교 테이블 생성
    missing_comparison = missing_before.merge(missing_after, on='Feature')

    # 결측치 비교 결과를 깔끔하게 출력
    display(missing_comparison)

## '동+아파트명' 기준으로 train + add 진행

- 원래는 '아파트명' 기준으로 진행하려 하였으나, '동'을 활용한 이유는, 같은 이름의 아파트가 존재할 수 있다고 판단하여, '동+아파트명'으로 진행함.
- 같은 아파트의 '층'에 따라서도 거래가격이 다르니, '층'을 함께 넣어야 하나, add 데이터에는 '층'이 없어서, '층'은 넣지 못함.
- 본 과정을 통해, 피처별로 결측치가 소폭 줄어 들었음.

In [58]:
# 새로운 피처 '동+아파트명' 생성
test['동+아파트명'] = test['동'] + ' ' + test['아파트명']
add['동+아파트명'] = add['주소(읍면동)'] + ' ' + add['k-아파트명']

missing_update('동+아파트명')

,Feature,Missing_Before,Missing_Before_%,Missing_After,Missing_After_%
0,k-전화번호,6576,70.923210,6441,69.467213
1,k-팩스번호,6606,71.246764,6475,69.833909
2,단지소개기존clob,8718,94.025022,8682,93.636756
3,k-세대타입(분양형태),6562,70.772217,6427,69.316221
4,k-관리방식,6562,70.772217,6427,69.316221
5,k-복도유형,6564,70.793788,6429,69.337791
6,k-난방방식,6562,70.772217,6427,69.316221
7,k-전체동수,6577,70.933995,6442,69.477998
8,k-전체세대수,6562,70.772217,6425,69.294651
9,k-건설사(시공사),6579,70.955565,6447,69.531924


## '도로명' 기준으로 train + add 진행

- train 데이터의 '도로명' 피처와 동일한 형태로 add에도 피처 생성함.
- 이를 두 데이터 합치는 key 값으로 사용함.
- 결측치가 약 69% 정도인 피처들이 대부분 약 15% 정도로 많이 줄어듬.

In [59]:
add['도로명'] = add['주소(도로명)'] + ' ' + add['주소(도로상세주소)']

missing_update('도로명')

,Feature,Missing_Before,Missing_Before_%,Missing_After,Missing_After_%
0,k-전화번호,6441,69.467213,1460,15.746333
1,k-팩스번호,6475,69.833909,1547,16.684642
2,단지소개기존clob,8682,93.636756,7698,83.024159
3,k-세대타입(분양형태),6427,69.316221,1467,15.821829
4,k-관리방식,6427,69.316221,1467,15.821829
5,k-복도유형,6429,69.337791,1467,15.821829
6,k-난방방식,6427,69.316221,1459,15.735548
7,k-전체동수,6442,69.477998,1467,15.821829
8,k-전체세대수,6425,69.294651,1457,15.713978
9,k-건설사(시공사),6447,69.531924,1489,16.059103


## GPS 좌표 기준으로 지하철 정보 가공

In [ ]:
from scipy.spatial import cKDTree

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # 지구의 반경 (km)
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = R * c
    return distance * 1000  # 미터 단위로 변환

def walking_time(distance):
    return distance / (4000/60)  # 4km/h의 걷는 속도 가정

def add_subway_features(apartment_df, subway_df):
    apartment_coords = apartment_df[['좌표Y', '좌표X']].values
    station_coords = subway_df[['위도', '경도']].values

    # 가장 가까운 3개의 역 찾기
    tree = cKDTree(station_coords)
    distances, indices = tree.query(apartment_coords, k=3)

    for i in range(1):
        apartment_df[f'{i+1}번째_가까운_역_이름'] = subway_df.loc[indices[:, i], '역사명'].values
        apartment_df[f'{i+1}번째_가까운_역_호선'] = subway_df.loc[indices[:, i], '호선'].values
        apartment_df[f'{i+1}번째_가까운_역_거리'] = np.array([haversine_distance(ac[0], ac[1], station_coords[idx][0], station_coords[idx][1]) 
                                                   for ac, idx in zip(apartment_coords, indices[:, i])])
        # apartment_df[f'{i+1}번째_가까운_역_도보시간'] = walking_time(apartment_df[f'{i+1}번째_가까운_역_거리'])

    # 시간대별 역 개수 계산
    def count_stations_in_time_range(min_time, max_time):
        min_dist = min_time * (4000/60)
        max_dist = max_time * (4000/60)
        return np.array([np.sum((min_dist < haversine_distance(c[0], c[1], station_coords[:, 0], station_coords[:, 1])) & 
                                (haversine_distance(c[0], c[1], station_coords[:, 0], station_coords[:, 1]) <= max_dist)) 
                         for c in apartment_coords])

    apartment_df['5분이하_역_개수'] = count_stations_in_time_range(0, 5)
    apartment_df['5분초과_10분이하_역_개수'] = count_stations_in_time_range(5, 10)
    # apartment_df['10분초과_15분이하_역_개수'] = count_stations_in_time_range(10, 15)
    # apartment_df['15분초과_20분이하_역_개수'] = count_stations_in_time_range(15, 20)

    return apartment_df

test = add_subway_features(test, subway)
test['1번째_가까운_역_이름'] = test['1번째_가까운_역_이름'].astype('category')
test['1번째_가까운_역_호선'] = test['1번째_가까운_역_호선'].astype('category')
test['1번째_가까운_역_이름'] = test['1번째_가까운_역_이름'].astype('category')

## 최종 확인

In [61]:
test.isna().mean() * 100

시군구                        0.000000
번지                         0.000000
본번                         0.000000
부번                         0.000000
아파트명                       0.000000
전용면적(㎡)                    0.000000
계약년월                       0.000000
계약일                        0.000000
층                          0.000000
건축년도                       0.000000
도로명                        0.000000
해제사유발생일                   97.713546
등기신청일자                     0.000000
거래유형                       0.000000
중개사소재지                     0.000000
k-단지분류(아파트,주상복합등등)        70.987921
k-전화번호                    15.746333
k-팩스번호                    16.684642
단지소개기존clob                83.024159
k-세대타입(분양형태)              15.821829
k-관리방식                    15.821829
k-복도유형                    15.821829
k-난방방식                    15.735548
k-전체동수                    15.821829
k-전체세대수                   15.713978
k-건설사(시공사)                16.059103
k-시행사                     16.102243
k-사용검사일-사용승인일             15

In [60]:
# 파일 저장 부분이기 때문에, 실제 저장할 경우만 주석 풀고 사용!
test.to_csv('data/kkh_test.csv', index=False)